## Fast Api

In [ ]:
# Get request

@app.get("/")
def read_root():
    return {"Hello": "World"}

## Post Request 

In [ ]:
# Firstly when we use post reqest it help us to create a new data in the server.
# Simple post request which have no data than it behave like get but we need to send and receive data 
# that's why se use post request. So we send the data in the body of the request and we receive the data in the body of the response.

@app.post("/items/")
def create_item():
    return {"hello" : "world"}


Data Extract from the body of the payload

In [ ]:
@app.post("/add")
def create_post(payload: dict = Body(...)):
    print(payload)
    return {"message": "Data received"}

Here we extract the data and send it back from the request

In [ ]:
# how to send data in a body of a post request and also extract that data from the body.
# normally we store that data in the database.
@app.post("/new_post")
def create_post(payload: dict = Body(...)):
    print(payload)
    return {"new_post": f"title {payload['title']} content: {payload['content']}"}

# Here issue is that client send any data they want to send. So we need to validate the data before storing it in the database.
# that's why we use pydantic model to validate the data.

Why we need schema 

It's pain to get all the values from the body. <br>
The client can send whatever data they want. <br>
The data isn't getting validated. <br>
We ultimately want to force the client to send data in a schema that we expect. <br>

For that we use library called pydantic it validate our data .

In [ ]:
class Post(BaseModel):
    title: str
    content: str
    published: bool = True
    rating: Optional[int] = None

@app.post("/new_post")
def create_post(new_post:Post):
    print(new_post)
    return{"new" :"post_created"}

We use pydantic model_dump/ dict to convert the body data into dictionary

In [ ]:
# Now work with api to retrieve post 
# In fast api when we send array data to it it gonna serialize it on way means it convert 
# array into a json format.

new_posts = [{"title": "post1", "content": "content1","id":1},
             {"title": "post2", "content": "content2","id":2}]

@app.get("/posts")
def get_posts():
    return {"posts": new_posts}

In [ ]:
# This method is used to update the post in the database. Here we have no db that's why we 
# use list to append it with unique id.

@app.post("/new_posts")
def create_new_post(post: Post):
    post_dict = post.dict()
    post_dict['id'] = randrange(0,100000)
    new_posts.append(post_dict)
    print(new_posts)
    return {"data": post_dict}

### Path parameter

In [ ]:

@app.get("/posts/{post_id}")
def get_post(post_id: int):
    return {"post": f"here is post {post_id}"}

In [ ]:
# no need to hard code the error value use http exception to handle the error.


@app.get("/posts/{post_id}")
def get_post(post_id: int , response: Response):
    post = find_post(post_id)
    if not post:
        raise HTTPException(status_code=status.HTTP_404_NOT_FOUND, 
                            detail=f"post with id {post_id} not found")
        # response.status_code = status.HTTP_404_NOT_FOUND
        # return {"detail": f"post with id {post_id} not found"}
    return {"post": f"here is post {post}"}

## Delete Post

In [ ]:
@app.delete("/posts/{post_id}")
def post_delete(post_id: int):
    index = find_index_post(post_id)    
    new_posts.pop(index) # type: ignore
    return {"message": "post deleted successfully"}

here we use http 204 status code to show that the post is deleted successfully. But this error msg
is not returned any message.

@app.delete("/posts/{post_id}")
def post_delete(post_id: int):
    index = find_index_post(post_id)    
    new_posts.pop(index) # type: ignore
    return Response(status_code=status.HTTP_204_NO_CONTENT)

## Update Method (PUT)

In [ ]:
# In this put mehtod we receive data from the front end we store it in post variable and then 
# we find the index of the post and then we update the post in the list.

@app.put("/posts/{post_id}")
def update_post(post_id: int, post: Post):
    index = find_index_post(post_id)
    if index is None:
        raise HTTPException(status_code=status.HTTP_404_NOT_FOUND, 
                            detail=f"post with id: {post_id} not found")
    post_dict = post.dict()
    post_dict['id'] = post_id
    new_posts[index] = post_dict
    return {"message": "post updated successfully"}


## Database (postgres)



Here we can established a new connection to the database.

In [ ]:
# This is the hard code value which is given which is not a good practice so we can convert in to 
# dynamic value.

    try:
        conn = psycopg2.connect(host="localhost",database="fastapi",user="postgres",
                                password="Pk135430",cursor_factory=RealDictCursor)
        curr = conn.cursor()
        print("Database connected successfully")
        break
    except Exception as e:
        print("Database connection error")
        print(f"Error: {e}")
        time.sleep(5)


Here we can enter data into the database and than commit it.

In [ ]:
@app.post("/new_posts", status_code= status.HTTP_201_CREATED)
def create_new_post(post: Post):
    
    # This is work but this can cause a sql injection probelem which attacker is used.
    # curr.execute(f"""INSERT INTO post (title, content) VALUES ('{post.title}', '{post.content}')""")
    # so we use
    curr.execute("""INSERT INTO post (title, content,published) VALUES (%s, %s,%s) 
                 RETURNING *""",(post.title, post.content,post.published))
    new_post = curr.fetchone()
    conn.commit()
    return {"new_post": new_post}
    

Fetching an individual by an id